In [1]:
import pandas as pd
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from sqlalchemy import create_engine
from pandas import json_normalize 

In [2]:
es = Elasticsearch(
    ['http://eswebportalrossko.rossko.local:9200/'],
    http_auth=('log', 'pass'),
    timeout=100,
    max_retries=10,
    retry_on_timeout=True
)

In [4]:
filial = ['Екатеринбург']
start_date = "2024-01-01"
control_date = "2024-01-03"

In [5]:
#Есть продукт и кросс
pcv_df_1=pd.DataFrame()
result =es.search(index="cart_item_event-*", 
              body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ]
                          }
              },
                    "aggs":{
                        "my_buckets":{
                            "composite":{
                                "sources":[
                                    {
                                        "filial":{
                                            "terms":{
                                                "field":"region"
                                            }
                                        }
                                    },
                                    {
                                        "member_id":{
                                            "terms":{
                                                "field":"member_id"
                                            }
                                        }
                                    },
                                    {
                                        "search_uid":{
                                            "terms":{
                                                "field":"search_uid.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "nsi":{
                                            "terms":{
                                                "field":"nsi.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_product":{
                                            "terms":{
                                                "field":"delivery_types.product.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_crosses":{
                                            "terms":{
                                                "field":"delivery_types.crosses.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "date":{
                                            "date_histogram": {
                                                "field": "server_timestamp",
                                                "calendar_interval": "1d",
                                                "format": "yyyy-MM-dd"
                                            }
                                        }
                                    }
                                ],
                                "size" : 1000
                            }
                        }
                    }
                   }
             )
g = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
pcv_df_1 = pd.concat([pcv_df_1,g], join = 'outer', ignore_index=True)
while 'after_key' in list(result['aggregations']['my_buckets'].keys()):
    k = result['aggregations']['my_buckets']['after_key']
    result = es.search(index="cart_item_event-*", 
                  body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ]
                          }
              },
                        "aggs":{
                            "my_buckets":{
                                "composite":{
                                    "sources":[
                                        {
                                            "filial":{
                                                "terms":{
                                                    "field":"region"
                                                }
                                            }
                                        },
                                        {
                                            "member_id":{
                                                "terms":{
                                                    "field":"member_id"
                                                }
                                            }
                                        },
                                        {
                                            "search_uid":{
                                                "terms":{
                                                    "field":"search_uid.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "nsi":{
                                                "terms":{
                                                    "field":"nsi.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_product":{
                                                "terms":{
                                                    "field":"delivery_types.product.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_crosses":{
                                                "terms":{
                                                    "field":"delivery_types.crosses.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "date":{
                                                "date_histogram": {
                                                    "field": "server_timestamp",
                                                    "calendar_interval": "1d",
                                                    "format": "yyyy-MM-dd"
                                                }
                                            }
                                        }
                                    ],
                                    "size" : 1000,
                                    "after": k
                                }
                            }
                        }
                       }
                 )
    docs = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
    pcv_df_1=pd.concat([pcv_df_1,docs], join = 'outer', ignore_index=True)

C:\Users\azubi\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: this request accesses system indices: [.kibana_-1310716657__1, .kibana_-1467765373_estest_2, .kibana_-152937574_admintenant_2, .kibana_-1596654582__1, .kibana_-1604638111_phpportal_1, .kibana_-1910296146__1, .kibana_-355631268_eswebservice_2, .kibana_-502342359__1, .kibana_-509616255_analyticparterra_1, .kibana_-709612266_zabbix_1, .kibana_-786847303__1, .kibana_1352823062__1, .kibana_1524852814__1, .kibana_2, .kibana_2069879251_vsqlolapreports_1, .kibana_3556498_test_1, .kibana_749487881_analyticaggregator_1, .kibana_804307555__1, .kibana_863498610__1, .kibana_92668751_admin_3], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
C:\Users\azubi\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:193:

In [6]:
#Нет ни продукта, ни кросса
pcv_df_2=pd.DataFrame()
result =es.search(index="cart_item_event-*", 
              body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  },
                              ]
                          }
              },
                    "aggs":{
                        "my_buckets":{
                            "composite":{
                                "sources":[
                                    {
                                        "filial":{
                                            "terms":{
                                                "field":"region"
                                            }
                                        }
                                    },
                                    {
                                        "member_id":{
                                            "terms":{
                                                "field":"member_id"
                                            }
                                        }
                                    },
                                    {
                                        "search_uid":{
                                            "terms":{
                                                "field":"search_uid.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "nsi":{
                                            "terms":{
                                                "field":"nsi.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "date":{
                                            "date_histogram": {
                                                "field": "server_timestamp",
                                                "calendar_interval": "1d",
                                                "format": "yyyy-MM-dd"
                                            }
                                        }
                                    }
                                ],
                                "size" : 1000
                            }
                        }
                    }
                   }
             )
g = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
pcv_df_2 = pd.concat([pcv_df_2,g], join = 'outer', ignore_index=True)
while 'after_key' in list(result['aggregations']['my_buckets'].keys()):
    k = result['aggregations']['my_buckets']['after_key']
    result = es.search(index="cart_item_event-*", 
                  body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  },
                              ]
                          }
              },
                        "aggs":{
                            "my_buckets":{
                                "composite":{
                                    "sources":[
                                        {
                                            "filial":{
                                                "terms":{
                                                    "field":"region"
                                                }
                                            }
                                        },
                                        {
                                            "member_id":{
                                                "terms":{
                                                    "field":"member_id"
                                                }
                                            }
                                        },
                                        {
                                            "search_uid":{
                                                "terms":{
                                                    "field":"search_uid.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "nsi":{
                                                "terms":{
                                                    "field":"nsi.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "date":{
                                                "date_histogram": {
                                                    "field": "server_timestamp",
                                                    "calendar_interval": "1d",
                                                    "format": "yyyy-MM-dd"
                                                }
                                            }
                                        }
                                    ],
                                    "size" : 1000,
                                    "after": k
                                }
                            }
                        }
                       }
                 )
    docs = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
    pcv_df_2=pd.concat([pcv_df_2,docs], join = 'outer', ignore_index=True)

C:\Users\azubi\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: this request accesses system indices: [.kibana_-1310716657__1, .kibana_-1467765373_estest_2, .kibana_-152937574_admintenant_2, .kibana_-1596654582__1, .kibana_-1604638111_phpportal_1, .kibana_-1910296146__1, .kibana_-355631268_eswebservice_2, .kibana_-502342359__1, .kibana_-509616255_analyticparterra_1, .kibana_-709612266_zabbix_1, .kibana_-786847303__1, .kibana_1352823062__1, .kibana_1524852814__1, .kibana_2, .kibana_2069879251_vsqlolapreports_1, .kibana_3556498_test_1, .kibana_749487881_analyticaggregator_1, .kibana_804307555__1, .kibana_863498610__1, .kibana_92668751_admin_3], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
C:\Users\azubi\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:193:

In [7]:
#Нет продукта, есть кросс
pcv_df_3=pd.DataFrame()
result =es.search(index="cart_item_event-*", 
              body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  }
                              ]
                          }
              },
                    "aggs":{
                        "my_buckets":{
                            "composite":{
                                "sources":[
                                    {
                                        "filial":{
                                            "terms":{
                                                "field":"region"
                                            }
                                        }
                                    },
                                    {
                                        "member_id":{
                                            "terms":{
                                                "field":"member_id"
                                            }
                                        }
                                    },
                                    {
                                        "search_uid":{
                                            "terms":{
                                                "field":"search_uid.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "nsi":{
                                            "terms":{
                                                "field":"nsi.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_crosses":{
                                            "terms":{
                                                "field":"delivery_types.crosses.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "date":{
                                            "date_histogram": {
                                                "field": "server_timestamp",
                                                "calendar_interval": "1d",
                                                "format": "yyyy-MM-dd"
                                            }
                                        }
                                    }
                                ],
                                "size" : 1000
                            }
                        }
                    }
                   }
             )
g = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
pcv_df_3 = pd.concat([pcv_df_3,g], join = 'outer', ignore_index=True)
while 'after_key' in list(result['aggregations']['my_buckets'].keys()):
    k = result['aggregations']['my_buckets']['after_key']
    result = es.search(index="cart_item_event-*", 
                  body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  }
                              ]
                          }
              },
                        "aggs":{
                            "my_buckets":{
                                "composite":{
                                    "sources":[
                                        {
                                            "filial":{
                                                "terms":{
                                                    "field":"region"
                                                }
                                            }
                                        },
                                        {
                                            "member_id":{
                                                "terms":{
                                                    "field":"member_id"
                                                }
                                            }
                                        },
                                        {
                                            "search_uid":{
                                                "terms":{
                                                    "field":"search_uid.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "nsi":{
                                                "terms":{
                                                    "field":"nsi.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_crosses":{
                                                "terms":{
                                                    "field":"delivery_types.crosses.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "date":{
                                                "date_histogram": {
                                                    "field": "server_timestamp",
                                                    "calendar_interval": "1d",
                                                    "format": "yyyy-MM-dd"
                                                }
                                            }
                                        }
                                    ],
                                    "size" : 1000,
                                    "after": k
                                }
                            }
                        }
                       }
                 )
    docs = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
    pcv_df_3=pd.concat([pcv_df_3,docs], join = 'outer', ignore_index=True)

C:\Users\azubi\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: this request accesses system indices: [.kibana_-1310716657__1, .kibana_-1467765373_estest_2, .kibana_-152937574_admintenant_2, .kibana_-1596654582__1, .kibana_-1604638111_phpportal_1, .kibana_-1910296146__1, .kibana_-355631268_eswebservice_2, .kibana_-502342359__1, .kibana_-509616255_analyticparterra_1, .kibana_-709612266_zabbix_1, .kibana_-786847303__1, .kibana_1352823062__1, .kibana_1524852814__1, .kibana_2, .kibana_2069879251_vsqlolapreports_1, .kibana_3556498_test_1, .kibana_749487881_analyticaggregator_1, .kibana_804307555__1, .kibana_863498610__1, .kibana_92668751_admin_3], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
C:\Users\azubi\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:193:

In [8]:
#Есть продукт, нет кросса
pcv_df_4=pd.DataFrame()
result =es.search(index="cart_item_event-*", 
              body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  }
                              ]
                          }
              },
                    "aggs":{
                        "my_buckets":{
                            "composite":{
                                "sources":[
                                    {
                                        "filial":{
                                            "terms":{
                                                "field":"region"
                                            }
                                        }
                                    },
                                    {
                                        "member_id":{
                                            "terms":{
                                                "field":"member_id"
                                            }
                                        }
                                    },
                                    {
                                        "search_uid":{
                                            "terms":{
                                                "field":"search_uid.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "nsi":{
                                            "terms":{
                                                "field":"nsi.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_product":{
                                            "terms":{
                                                "field":"delivery_types.product.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "date":{
                                            "date_histogram": {
                                                "field": "server_timestamp",
                                                "calendar_interval": "1d",
                                                "format": "yyyy-MM-dd"
                                            }
                                        }
                                    }
                                ],
                                "size" : 1000
                            }
                        }
                    }
                   }
             )
g = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
pcv_df_4 = pd.concat([pcv_df_4,g], join = 'outer', ignore_index=True)
while 'after_key' in list(result['aggregations']['my_buckets'].keys()):
    k = result['aggregations']['my_buckets']['after_key']
    result = es.search(index="cart_item_event-*", 
                  body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  }
                              ]
                          }
              },
                        "aggs":{
                            "my_buckets":{
                                "composite":{
                                    "sources":[
                                        {
                                            "filial":{
                                                "terms":{
                                                    "field":"region"
                                                }
                                            }
                                        },
                                        {
                                            "member_id":{
                                                "terms":{
                                                    "field":"member_id"
                                                }
                                            }
                                        },
                                        {
                                            "search_uid":{
                                                "terms":{
                                                    "field":"search_uid.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "nsi":{
                                                "terms":{
                                                    "field":"nsi.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_product":{
                                                "terms":{
                                                    "field":"delivery_types.product.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "date":{
                                                "date_histogram": {
                                                    "field": "server_timestamp",
                                                    "calendar_interval": "1d",
                                                    "format": "yyyy-MM-dd"
                                                }
                                            }
                                        }
                                    ],
                                    "size" : 1000,
                                    "after": k
                                }
                            }
                        }
                       }
                 )
    docs = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
    pcv_df_4=pd.concat([pcv_df_4,docs], join = 'outer', ignore_index=True)

C:\Users\azubi\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: this request accesses system indices: [.kibana_-1310716657__1, .kibana_-1467765373_estest_2, .kibana_-152937574_admintenant_2, .kibana_-1596654582__1, .kibana_-1604638111_phpportal_1, .kibana_-1910296146__1, .kibana_-355631268_eswebservice_2, .kibana_-502342359__1, .kibana_-509616255_analyticparterra_1, .kibana_-709612266_zabbix_1, .kibana_-786847303__1, .kibana_1352823062__1, .kibana_1524852814__1, .kibana_2, .kibana_2069879251_vsqlolapreports_1, .kibana_3556498_test_1, .kibana_749487881_analyticaggregator_1, .kibana_804307555__1, .kibana_863498610__1, .kibana_92668751_admin_3], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
C:\Users\azubi\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:193:

In [9]:
#Заполняем поля в неполных датасетах, устанавливаем единый порядок полей для конкатенации
pcv_df_2['key.delivery_types_product'] = 'Нет в наличии'
pcv_df_2['key.delivery_types_crosses'] = 'Нет в наличии'
pcv_df_3['key.delivery_types_product'] = 'Нет в наличии'
pcv_df_4['key.delivery_types_crosses'] = 'Нет в наличии'

pcv_df_1 = pcv_df_1[['key.filial','key.member_id','key.search_uid','key.nsi','key.delivery_types_product','key.delivery_types_crosses','key.date']]
pcv_df_2 = pcv_df_2[['key.filial','key.member_id','key.search_uid','key.nsi','key.delivery_types_product','key.delivery_types_crosses','key.date']]
pcv_df_3 = pcv_df_3[['key.filial','key.member_id','key.search_uid','key.nsi','key.delivery_types_product','key.delivery_types_crosses','key.date']]
pcv_df_4 = pcv_df_4[['key.filial','key.member_id','key.search_uid','key.nsi','key.delivery_types_product','key.delivery_types_crosses','key.date']]

In [10]:
pcv_df_final = pd.concat([pcv_df_1,
                          pcv_df_2,
                          pcv_df_3,
                          pcv_df_4],ignore_index=True)

In [11]:
#Выгружаем добавления в корзину с признаком источника nsi, типом склада
add_to_cart=pd.DataFrame()
result =es.search(index="cart_item_event-*", 
              body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['add_to_cart']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ]
                          }
              },
                    "aggs":{
                        "my_buckets":{
                            "composite":{
                                "sources":[
                                    {
                                        "member_id":{
                                            "terms":{
                                                "field":"member_id"
                                            }
                                        }
                                    },
                                    {
                                        "search_uid":{
                                            "terms":{
                                                "field":"search_uid.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "nsi":{
                                            "terms":{
                                                "field":"nsi.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_tsp":{
                                            "terms":{
                                                "field":"is_approximate_delivery_interval"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_local":{
                                            "terms":{
                                                "field":"stock.is_local_delivery"
                                            }
                                        }
                                    },
                                    {
                                        "product":{
                                            "terms":{
                                                "field":"product.path.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "date":{
                                            "date_histogram": {
                                                "field": "server_timestamp",
                                                "calendar_interval": "1d",
                                                "format": "yyyy-MM-dd"
                                            }
                                        }
                                    }
                                ],
                                "size" : 1000
                            }
                        }
                    }
                   }
             )
g = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
add_to_cart = pd.concat([add_to_cart,g], join = 'outer', ignore_index=True)
while 'after_key' in list(result['aggregations']['my_buckets'].keys()):
    k = result['aggregations']['my_buckets']['after_key']
    result = es.search(index="cart_item_event-*", 
                  body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['add_to_cart']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ]
                          }
              },
                        "aggs":{
                            "my_buckets":{
                                "composite":{
                                    "sources":[
                                        {
                                            "member_id":{
                                                "terms":{
                                                    "field":"member_id"
                                                }
                                            }
                                        },
                                        {
                                            "search_uid":{
                                                "terms":{
                                                    "field":"search_uid.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "nsi":{
                                                "terms":{
                                                    "field":"nsi.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_tsp":{
                                                "terms":{
                                                    "field":"is_approximate_delivery_interval"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_local":{
                                                "terms":{
                                                    "field":"stock.is_local_delivery"
                                                }
                                            }
                                        },
                                        {
                                            "product":{
                                                "terms":{
                                                    "field":"product.path.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "date":{
                                                "date_histogram": {
                                                    "field": "server_timestamp",
                                                    "calendar_interval": "1d",
                                                    "format": "yyyy-MM-dd"
                                                }
                                            }
                                        }
                                    ],
                                    "size" : 1000,
                                    "after": k
                                }
                            }
                        }
                       }
                 )
    docs = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
    add_to_cart=pd.concat([add_to_cart,docs], join = 'outer', ignore_index=True)

C:\Users\azubi\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: this request accesses system indices: [.kibana_-1310716657__1, .kibana_-1467765373_estest_2, .kibana_-152937574_admintenant_2, .kibana_-1596654582__1, .kibana_-1604638111_phpportal_1, .kibana_-1910296146__1, .kibana_-355631268_eswebservice_2, .kibana_-502342359__1, .kibana_-509616255_analyticparterra_1, .kibana_-709612266_zabbix_1, .kibana_-786847303__1, .kibana_1352823062__1, .kibana_1524852814__1, .kibana_2, .kibana_2069879251_vsqlolapreports_1, .kibana_3556498_test_1, .kibana_749487881_analyticaggregator_1, .kibana_804307555__1, .kibana_863498610__1, .kibana_92668751_admin_3], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
C:\Users\azubi\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:193:

In [12]:
#Выделяем НСИ на карточке, из которой было добавление в корзину
add_to_cart['key.product']=add_to_cart['key.product'].astype(str).map(lambda x: x.lstrip('/product?text='))

In [13]:
#Разграничение НСИ карточки и корзины, подготовка мерджа карточки и корзины
add_to_cart.rename(columns={'key.nsi':'key.nsi_cart'},inplace=True)
add_to_cart.rename(columns={'key.product':'key.nsi'},inplace=True)
add_to_cart['is_cart']=1

In [14]:
new_df = pcv_df_final.merge(add_to_cart[['key.member_id',
                                         'key.search_uid',
                                         'key.nsi',
                                         'key.delivery_types_tsp',
                                         'key.delivery_types_local',
                                         'is_cart']], how='left', on = ['key.member_id',
                                                                     'key.search_uid',
                                                                     'key.nsi'])


In [15]:
new_df['is_cart'].fillna(0,inplace=True)
new_df['key.delivery_types_tsp'].fillna(False,inplace=True)
new_df['key.delivery_types_local'].fillna(False,inplace=True)

In [16]:
new_df

,key.filial,key.member_id,key.search_uid,key.nsi,key.delivery_types_product,key.delivery_types_crosses,key.date,key.delivery_types_tsp,key.delivery_types_local,is_cart
0,Екатеринбург,8160,020e3f6c-44a8-46bc-b718-b1b1c0ad5d09,NSIN0023169008,ТСП/Росско,ТСП/Росско,2024-01-02,False,True,1.0
1,Екатеринбург,8160,17f229cb-b28c-4f89-8415-6d31623e9256,NSIN0023169008,ТСП/Росско,ТСП/Росско,2024-01-02,False,True,1.0
2,Екатеринбург,8160,180ac1b2-ab3d-4b4d-be2b-384129da10d5,NSII0007955452,ТСП/Росско,ТСП/Росско,2024-01-02,False,True,1.0
3,Екатеринбург,8160,2d64dc4e-3ebe-4ea1-a251-ced326eb45af,NSII0007955452,ТСП/Росско,ТСП/Росско,2024-01-02,False,False,0.0
4,Екатеринбург,8160,3f98f53b-aeb2-4626-8b6a-248b8178471a,NSII0023283917,Росско,ТСП/Росско,2024-01-02,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...
2510,Екатеринбург,219521,84e55080-e2ef-401a-abe0-43271069212a,NSIN0003484006,Росско,Нет в наличии,2024-01-02,False,False,0.0
2511,Екатеринбург,222604,15e05b11-f37d-400e-98bb-4e7ea9e47fb7,NSII0013019625,ТСП,Нет в наличии,2024-01-02,False,False,0.0
2512,Екатеринбург,227304,5eb7fa74-a065-4e56-8c78-42e60c4188de,NSIN0021121958,Росско,Нет в наличии,2024-01-01,False,False,0.0
2513,Екатеринбург,230143,a56c5078-0a6e-464b-a6d6-6b371d4f94e5,NSIN0011325424,Росско,Нет в наличии,2024-01-01,False,False,0.0


In [17]:
new_df['key.delivery_types_crosses'].value_counts()

ТСП/Росско       1737
Нет в наличии     431
ТСП               279
Росско             68
Name: key.delivery_types_crosses, dtype: int64

In [18]:
new_df[['key.delivery_types_product','key.delivery_types_crosses','key.delivery_types_tsp','key.delivery_types_local']].value_counts()

key.delivery_types_product  key.delivery_types_crosses  key.delivery_types_tsp  key.delivery_types_local
ТСП                         ТСП/Росско                  False                   False                       401
Росско                      ТСП/Росско                  False                   False                       303
Нет в наличии               ТСП/Росско                  False                   False                       257
                            Нет в наличии               False                   False                       222
ТСП/Росско                  ТСП/Росско                  False                   True                        218
                                                                                False                       199
Росско                      ТСП/Росско                  False                   True                        194
ТСП                         Нет в наличии               False                   False                       144

In [19]:
new_df.loc[new_df['is_cart']==1,['key.delivery_types_product','key.delivery_types_crosses','key.delivery_types_tsp','key.delivery_types_local']].value_counts()

key.delivery_types_product  key.delivery_types_crosses  key.delivery_types_tsp  key.delivery_types_local
ТСП/Росско                  ТСП/Росско                  False                   True                        218
Росско                      ТСП/Росско                  False                   True                        194
ТСП                         ТСП/Росско                  False                   True                         89
Нет в наличии               ТСП/Росско                  False                   True                         48
ТСП/Росско                  ТСП/Росско                  False                   False                        20
ТСП                         ТСП/Росско                  True                    False                        19
                            Нет в наличии               True                    False                        17
                            ТСП                         True                    False                        15